# Goal

**Want to build a model that predicts the number of purchases a customer will make in a month based on the Recency-Frequency-Monetary Features**

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import datetime as dt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data

In [ ]:
online = pd.read_csv("./Data/online12m.csv",parse_dates=['InvoiceDate'],index_col=0)

In [ ]:
online['TotalSum'] = online.Quantity * online.UnitPrice
online['InvoiceMonth'] = online.InvoiceDate.dt.strftime("%Y-%m")

In [ ]:
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum,InvoiceMonth
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60,2011-10
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45,2011-11
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50,2011-07
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10,2011-11
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85,2011-05


# Data Pre-Processing and Feature Engineering

**Use number of purchases for a certain customer in the last month as the target variable.** 

In [ ]:
online.groupby(['InvoiceMonth']).size()

InvoiceMonth
2010-12     4893
2011-01     3580
2011-02     3648
2011-03     4764
2011-04     4148
2011-05     5018
2011-06     4669
2011-07     4610
2011-08     4744
2011-09     7189
2011-10     8808
2011-11    11644
dtype: int64

In [ ]:
# Build a pivot table counting invoices for each customer monthly
cust_month_tx = pd.pivot_table(data=online, values='InvoiceNo',
                               index=['CustomerID'], columns=['InvoiceMonth'],
                               aggfunc=pd.Series.nunique, fill_value=0)

cust_month_tx.head(2)

InvoiceMonth,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11
CustomerID,,,,,,,,,,,,
12747,2,1,0,1,0,2,1,0,1,0,1,1
12748,24,2,4,9,3,17,12,8,9,9,10,41


In [ ]:
y = cust_month_tx.loc[:,'2011-11']

In [ ]:
lagged_features = cust_month_tx.iloc[:,:-1]


**Build features using rfm characteristics** 

In [ ]:
online_X = online[online['InvoiceMonth']!='2011-11']

In [ ]:
# Define the snapshot date
NOW = dt.datetime(2011,11,1)

features = online_X.groupby('CustomerID').agg({
 
  'InvoiceDate': lambda x: (NOW - x.max()).days,
  'InvoiceNo': pd.Series.nunique,
  'TotalSum': np.sum,
  'Quantity': ['mean', 'sum']}).reset_index()

# Rename the columns
features.columns = ['CustomerID', 'recency', 'frequency', 'monetary', 'quantity_avg', 'quantity_total']

In [ ]:
features.head()

,CustomerID,recency,frequency,monetary,quantity_avg,quantity_total
0,12747,27,9,643.33,10.523810,221
1,12748,5,107,4576.23,6.727110,3747
2,12749,91,2,598.65,8.791667,211
3,12820,5,3,202.62,10.307692,134
4,12822,31,2,146.15,9.666667,87


Note: Have to remove new customers from the target variable

In [ ]:
y = y.loc[y.index.intersection(features.CustomerID)]

**Data Split**

In [ ]:
custid = ['CustomerID']
cols = [col for col in features.columns if col not in custid]

In [ ]:
X = features[cols]
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.25, random_state=99)

# Fitting and Evaluate the Model

In [ ]:
# Initialize linear regression instance
linreg = LinearRegression()
linreg.fit(train_X, train_Y)
test_pred_Y = linreg.predict(test_X)

In [ ]:
# Calculate root mean squared error on testing data
rmse_test = np.sqrt(mean_squared_error(test_Y, test_pred_Y))
print(f'RMSE test: {rmse_test}')

RMSE test: 1.2156885786624618


In [ ]:
np.std(test_Y)

1.656845129857455

# Inference Modeling

In [ ]:
train_X.corr()

,recency,frequency,monetary,quantity_avg,quantity_total
recency,1.000000,-0.246385,-0.123775,0.014336,-0.130374
frequency,-0.246385,1.000000,0.524829,0.005898,0.540954
monetary,-0.123775,0.524829,1.000000,0.064503,0.907913
quantity_avg,0.014336,0.005898,0.064503,1.000000,0.091078
quantity_total,-0.130374,0.540954,0.907913,0.091078,1.000000


Drop quantity_total because it is strongly correlated with monetary

**Fit model with only RFM**

In [ ]:
# Fit Model and Print Summary
olsreg = sm.OLS(train_Y.values, train_X.drop(['quantity_total'],axis=1))
olsreg = olsreg.fit()

print(olsreg.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.486
Model:                            OLS   Adj. R-squared (uncentered):              0.485
Method:                 Least Squares   F-statistic:                              597.4
Date:                Wed, 24 Nov 2021   Prob (F-statistic):                        0.00
Time:                        14:11:43   Log-Likelihood:                         -2773.1
No. Observations:                2529   AIC:                                      5554.
Df Residuals:                    2525   BIC:                                      5578.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In this model, frequency and monetary value appear to be significant while recency is significant only at .10 level. Also, the R^2 is relatively low, so its likely more features need to be built or gathered.

**Fit using lagged features**

In [ ]:
features_with_laggged = features.merge(lagged_features,on='CustomerID')
features_with_laggged.head()

,CustomerID,recency,frequency,monetary,quantity_avg,quantity_total,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10
0,12747,27,9,643.33,10.523810,221,2,1,0,1,0,2,1,0,1,0,1
1,12748,5,107,4576.23,6.727110,3747,24,2,4,9,3,17,12,8,9,9,10
2,12749,91,2,598.65,8.791667,211,0,0,0,0,0,1,0,0,1,0,0
3,12820,5,3,202.62,10.307692,134,0,1,0,0,0,0,0,0,0,1,1
4,12822,31,2,146.15,9.666667,87,0,0,0,0,0,0,0,0,0,2,0


In [ ]:
X_with_lagged = features_with_laggged.drop(["CustomerID","quantity_total"],axis=1)
train_X, test_X, train_Y, test_Y = train_test_split(X_with_lagged, y, test_size=0.25, random_state=99)

In [ ]:
# Fit Model and Print Summary
olsreg = sm.OLS(train_Y.values, train_X)
olsreg = olsreg.fit()

print(olsreg.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.510
Model:                            OLS   Adj. R-squared (uncentered):              0.507
Method:                 Least Squares   F-statistic:                              186.9
Date:                Wed, 24 Nov 2021   Prob (F-statistic):                        0.00
Time:                        14:24:59   Log-Likelihood:                         -2713.5
No. Observations:                2529   AIC:                                      5455.
Df Residuals:                    2515   BIC:                                      5537.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [ ]:
# Fit Model and Print Summary
olsreg = sm.OLS(train_Y.values, train_X.drop(['recency','frequency'],axis=1))
olsreg = olsreg.fit()

print(olsreg.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.506
Model:                            OLS   Adj. R-squared (uncentered):              0.503
Method:                 Least Squares   F-statistic:                              198.2
Date:                Wed, 24 Nov 2021   Prob (F-statistic):                        0.00
Time:                        14:27:00   Log-Likelihood:                         -2723.6
No. Observations:                2529   AIC:                                      5473.
Df Residuals:                    2516   BIC:                                      5549.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------